### Proyecto: Aplicación de Árboles de Búsqueda para Seguridad Territorial

**Autores:** Erika Amores, Brandon Aguiar, Alicia Padilla, Israel López

**Fecha:** Enero 2024


**1. Introducción**
 Este proyecto implementa un sistema completo para el análisis de datos de seguridad territorial utilizando:

 1. **Árboles Binarios de Búsqueda (BST)** 

 2. **Algoritmos de ordenamiento comparativos** 

 3. **Modelos de Machine Learning** 

 4. **Visualizaciones interactivas** 

 5. **Aplicación web** 




In [ ]:

# Configuración del Entorno
# Instalación de dependencias
!pip install flask pandas numpy plotly scikit-learn joblib matplotlib -q

# Importación de bibliotecas
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Añadir ruta de módulos
sys.path.append('estructuras')
sys.path.append('procesamiento')
sys.path.append('visualizacion')


# Verificar estructura del proyecto
print("Estructura del proyecto:")
for root, dirs, files in os.walk('.'):
    level = root.replace('.', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        if file.endswith('.py') or file.endswith('.csv'):
            print(f'{subindent}{file}')



Estructura del proyecto:
./
  ACLED_Ecuador_limpio.csv
  app.py
  excepciones.py
  .venv/
    Include/
    Lib/
      site-packages/
        decorator.py
        ipykernel_launcher.py
        ipython_pygments_lexers.py
        jupyter.py
        nest_asyncio.py
        pylab.py
        six.py
        threadpoolctl.py
        asttokens/
          astroid_compat.py
          asttokens.py
          line_numbers.py
          mark_tokens.py
          util.py
          version.py
          __init__.py
          __pycache__/
        asttokens-3.0.1.dist-info/
          licenses/
        blinker/
          base.py
          _utilities.py
          __init__.py
          __pycache__/
        blinker-1.9.0.dist-info/
        click/
          core.py
          decorators.py
          exceptions.py
          formatting.py
          globals.py
          parser.py
          shell_completion.py
          termui.py
          testing.py
          types.py
          utils.py
          _compat.py
        

In [ ]:

#  Carga y Limpieza de Datos

from procesamiento.limpieza_datos import LimpiezaDatos

# Cargar y limpiar datos (asumiendo que tienes ACLED_Ecuador.csv)
limpiador = LimpiezaDatos('datos/ACLED_Ecuador.csv')
df = limpiador.ejecutar_pipeline()

print(f" Datos procesados: {len(df)} registros")
print(f" Columnas disponibles: {list(df.columns)}")

# Mostrar primeras filas
df.head()


# Estadísticas descriptivas
estadisticas = limpiador.obtener_estadisticas()
print("=== ESTADÍSTICAS DEL DATASET ===")
print(f"Total registros: {estadisticas['registros_totales']}")
print(f"Total fatalidades: {estadisticas['fatalidades']['total']}")
print(f"Fatalidades promedio: {estadisticas['fatalidades']['promedio']:.2f}")
print(f"Máximo fatalidades: {estadisticas['fatalidades']['maximo']}")

# Top 5 provincias
print("\nTop 5 provincias con más eventos:")
for provincia, conteo in list(estadisticas['eventos_por_provincia'].items())[:5]:
    print(f"  {provincia}: {conteo} eventos")



Datos cargados: 4914 registros, 35 columnas
Registros de Ecuador: 4914
Pipeline completado: 4892 registros listos para ML
Datos limpios guardados en: ACLED_Ecuador_limpio.csv
 Datos procesados: 4892 registros
 Columnas disponibles: ['event_date', 'year', 'event_type', 'sub_event_type', 'actor1', 'actor2', 'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude', 'fatalities', 'notes', 'source', 'timestamp', 'valor_riesgo']
=== ESTADÍSTICAS DEL DATASET ===
Total registros: 4892
Total fatalidades: 4402
Fatalidades promedio: 0.90
Máximo fatalidades: 5

Top 5 provincias con más eventos:
  Guayas: 1647 eventos
  Manabi: 781 eventos
  El Oro: 734 eventos
  Los Rios: 405 eventos
  Esmeraldas: 319 eventos


In [ ]:

#  Implementación del Árbol Binario de Búsqueda


from estructuras.arbol_busqueda import ArbolSeguridadTerritorial
import time

# Crear árbol con fatalidades como clave
arbol = ArbolSeguridadTerritorial()

# Medir tiempo de inserción
inicio = time.time()

for _, fila in df.iterrows():
    clave = float(fila['fatalities'])
    arbol.insertar(clave, fila.to_dict())

tiempo_insercion = time.time() - inicio

print(f"  Tiempo de inserción: {tiempo_insercion:.4f} segundos")
print(f" Altura del árbol: {arbol.obtener_altura()}")


# Estadísticas del árbol
estadisticas_arbol = arbol.obtener_estadisticas()
print("=== ESTADÍSTICAS DEL ÁRBOL BST ===")
for clave, valor in estadisticas_arbol.items():
    print(f"{clave}: {valor}")


# Pruebas de búsqueda
print(" PRUEBAS DE BÚSQUEDA EN EL ÁRBOL")

# Búsqueda exacta
fatalidad_buscar = 0
resultados_exacta = arbol.buscar(fatalidad_buscar)
print(f"Eventos con {fatalidad_buscar} fatalidades: {len(resultados_exacta)}")

# Búsqueda por rango
min_fatal, max_fatal = 1, 5
resultados_rango = arbol.buscar_por_fatalidades(min_fatal, max_fatal)
print(f"Eventos con {min_fatal}-{max_fatal} fatalidades: {len(resultados_rango)}")

# Búsqueda por provincia
provincia_buscar = 'Pichincha'
resultados_provincia = arbol.buscar_por_provincia(provincia_buscar)
print(f"Eventos en {provincia_buscar}: {len(resultados_provincia)}")


  Tiempo de inserción: 0.2500 segundos
 Altura del árbol: 4
=== ESTADÍSTICAS DEL ÁRBOL BST ===
altura: 4
total_nodos: 6
total_registros: 4892
comparaciones_promedio: 1.6659852820932135
 PRUEBAS DE BÚSQUEDA EN EL ÁRBOL
Eventos con 0 fatalidades: 1830
Eventos con 1-5 fatalidades: 3062
Eventos en Pichincha: 228


In [ ]:

#  Análisis Comparativo de Algoritmos de Ordenamiento

from estructuras.algoritmos_ordenamiento import AnalisisOrdenamiento

# Extraer datos para ordenar
datos_fatalidades = df['fatalities'].dropna().astype(int).tolist()
datos_riesgo = df['valor_riesgo'].dropna().tolist()

print(f" Muestra para ordenamiento:")
print(f"  - Fatalidades: {len(datos_fatalidades)} valores (ej: {datos_fatalidades[:5]})")
print(f"  - Rango: {min(datos_fatalidades)} a {max(datos_fatalidades)}")


# Comparar algoritmos con datos de fatalidades (enteros)
analizador = AnalisisOrdenamiento()
resultados_fatalidades = analizador.comparar_algoritmos(datos_fatalidades, 'enteros')

print("=== RESULTADOS DE ORDENAMIENTO (FATALIDADES) ===")
for algoritmo, stats in resultados_fatalidades.items():
    if algoritmo != 'analisis':
        print(f"\n{algoritmo.replace('_', ' ').title()}:")
        print(f"  Tiempo: {stats['tiempo']*1000:.2f} ms")
        print(f"  Memoria: {stats['memoria_bytes']/1024:.1f} KB")
        print(f"  Complejidad: {stats['complejidad']}")


# Visualización comparativa
from visualizacion.graficos import VisualizadorSeguridad

fig = VisualizadorSeguridad.crear_grafico_comparacion_algoritmos(resultados_fatalidades)
fig.show()


 Muestra para ordenamiento:
  - Fatalidades: 4892 valores (ej: [1, 1, 1, 1, 1])
  - Rango: 0 a 5
=== RESULTADOS DE ORDENAMIENTO (FATALIDADES) ===

Counting Sort:
  Tiempo: 0.32 ms
  Memoria: 40.2 KB
  Complejidad: O(n + k)

Radix Sort:
  Tiempo: 1.58 ms
  Memoria: 76.7 KB
  Complejidad: O(d * (n + b))

Bucket Sort:
  Tiempo: 2.49 ms
  Memoria: 81.0 KB
  Complejidad: O(n + k) promedio


In [ ]:
from procesamiento.modelo_ml import ModeloSeguridad


# Crear y entrenar el modelo
modelo = ModeloSeguridad()
resultados_entrenamiento = modelo.entrenar_modelo(df, 'decision_tree')

print("=== RESULTADOS DEL MODELO ===")
print(f"Precisión (accuracy): {resultados_entrenamiento['accuracy'] * 100:.2f}%")
print(f"Tamaño del conjunto de entrenamiento: {resultados_entrenamiento['tamano_entrenamiento']}")
print(f"Tamaño del conjunto de prueba: {resultados_entrenamiento['tamano_prueba']}")

# Mostrar matriz de confusión
print("\nMatriz de confusión:")
for fila in resultados_entrenamiento['matriz_confusion']:
    print(fila)

# Ejemplo mínimo de predicción
datos_prueba = {
    'admin1': 'Guayas',
    'event_type': 'Protestas',
    'fatalities': 2
}

prediccion = modelo.predecir(datos_prueba)

print("\nEjemplo de predicción:")
print(f"Datos de entrada: {datos_prueba}")


print(resultados_entrenamiento)

print("Modelo entrenado:", modelo.modelo)
    



TypeError: ModeloSeguridad.entrenar_modelo() takes 2 positional arguments but 3 were given

In [ ]:
#  Visualizaciones con Plotly

from visualizacion.graficos import VisualizadorSeguridad

# Crear diferentes visualizaciones
print("Generando visualizaciones...")

# 1. Fatalidades por provincia
fig1 = VisualizadorSeguridad.crear_grafico_fatalidades_por_provincia(df)
fig1.show()

# 2. Tipos de evento
fig2 = VisualizadorSeguridad.crear_grafico_tipos_evento(df)
fig2.show()

# 3. Mapa 
if 'latitude' in df.columns and 'longitude' in df.columns:
    fig3 = VisualizadorSeguridad.crear_mapa_eventos(df)
    fig3.show()

# 4. Evolución temporal
fig4 = VisualizadorSeguridad.crear_grafico_evolucion_temporal(df)
fig4.show()

# 5. Estadísticas del árbol
fig5 = VisualizadorSeguridad.crear_visualizacion_arbol(estadisticas_arbol)
fig5.show()


Generando visualizaciones...


In [ ]:
#  Análisis de Eficiencia: BST vs Búsqueda Lineal


import time
import random

def busqueda_lineal(lista_datos, clave):
    """Búsqueda lineal O(n) para comparación"""
    resultados = []
    for dato in lista_datos:
        if dato.get('fatalities') == clave:
            resultados.append(dato)
    return resultados

# Preparar datos para comparación
datos_lista = [fila.to_dict() for _, fila in df.iterrows()]

# Medir tiempos para diferentes tamaños
tamanos = [100, 500, 1000, 5000]
tiempos_bst = []
tiempos_lineal = []

for n in tamanos:
    if n > len(datos_lista):
        break
    
    muestra = datos_lista[:n]
    
    # Construir árbol pequeño
    arbol_pequeno = ArbolSeguridadTerritorial()
    for dato in muestra:
        arbol_pequeno.insertar(dato['fatalities'], dato)
    
    # Búsqueda en BST
    inicio = time.time()
    arbol_pequeno.buscar(0)  
    tiempos_bst.append(time.time() - inicio)
    
    # Búsqueda lineal
    inicio = time.time()
    busqueda_lineal(muestra, 0)
    tiempos_lineal.append(time.time() - inicio)

# Crear gráfico comparativo
fig_comparacion = go.Figure()
fig_comparacion.add_trace(go.Scatter(
    x=tamanos[:len(tiempos_bst)],
    y=tiempos_bst,
    mode='lines+markers',
    name='BST (O(log n))',
    line=dict(color='green', width=3)
))
fig_comparacion.add_trace(go.Scatter(
    x=tamanos[:len(tiempos_lineal)],
    y=tiempos_lineal,
    mode='lines+markers',
    name='Búsqueda Lineal (O(n))',
    line=dict(color='red', width=3)
))

fig_comparacion.update_layout(
    title='Comparación: BST vs Búsqueda Lineal',
    xaxis_title='Tamaño del Dataset (registros)',
    yaxis_title='Tiempo de Búsqueda (segundos)',
    hovermode='x unified'
)

fig_comparacion.show()
